In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:

configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "#######",
"fs.azure.account.oauth2.client.secret": '#######',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/#######/oauth2/token"}


dbutils.fs.mount(
source = "abfss://containers@2001tokyoolympics.dfs.core.windows.net", # contrainer@storageacc  
mount_point = "/mnt/tokyoolymic",
extra_configs = configs)


True

In [ ]:
 %fs
 ls "/mnt/tokyoolymic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolymic/raw-data/,raw-data/,0,1720954814000
dbfs:/mnt/tokyoolymic/transformed-data/,transformed-data/,0,1720954834000


In [ ]:
spark

In [ ]:
athletes = spark.read.format('csv').option('header','true').option('inferSchema','true').load('/mnt/tokyoolymic/raw-data/athletes.csv')
coaches = spark.read.format('csv').option('header','true').option('inferSchema','true').load('/mnt/tokyoolymic/raw-data/coaches.csv')
entries_gender = spark.read.format('csv').option('header','true').option('inferSchema','true').load('/mnt/tokyoolymic/raw-data/entries-gender.csv')
medals = spark.read.format('csv').option('header','true').option('inferSchema','true').load('/mnt/tokyoolymic/raw-data/medals.csv')
teams = spark.read.format('csv').option('header','true').option('inferSchema','true').load('/mnt/tokyoolymic/raw-data/teams.csv')

In [ ]:
athletes.show(5)

+-----------------+------+-------------------+
|             Name|   NOC|         Discipline|
+-----------------+------+-------------------+
|  AALERUD Katrine|Norway|       Cycling Road|
|      ABAD Nestor| Spain|Artistic Gymnastics|
|ABAGNALE Giovanni| Italy|             Rowing|
|   ABALDE Alberto| Spain|         Basketball|
|    ABALDE Tamara| Spain|         Basketball|
+-----------------+------+-------------------+
only showing top 5 rows



In [ ]:
athletes.printSchema()

root
 |-- Name: string (nullable = true)
 |-- NOC: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [ ]:
coaches.show(5)

+---------------+-------------+----------+-----+
|           Name|          NOC|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| NULL|
|      ABE Junya|        Japan|Volleyball| NULL|
|  ABE Katsuhiko|        Japan|Basketball| NULL|
|   ADAMA Cherif|Côte d'Ivoire|  Football| NULL|
|     AGEBA Yuya|        Japan|Volleyball| NULL|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [ ]:
teams.show(5)

+-------+--------------+--------------------+-----+
|   Name|    Discipline|                 NOC|Event|
+-------+--------------+--------------------+-----+
|Belgium|3x3 Basketball|             Belgium|  Men|
|  China|3x3 Basketball|People's Republic...|  Men|
|  China|3x3 Basketball|People's Republic...|Women|
| France|3x3 Basketball|              France|Women|
|  Italy|3x3 Basketball|               Italy|Women|
+-------+--------------+--------------------+-----+
only showing top 5 rows



In [ ]:
entries_gender.show(5)

+-------------------+------+----+-----+
|         Discipline|Female|Male|Total|
+-------------------+------+----+-----+
|     3x3 Basketball|    32|  32|   64|
|            Archery|    64|  64|  128|
|Artistic Gymnastics|    98|  98|  196|
|  Artistic Swimming|   105|   0|  105|
|          Athletics|   969|1072| 2041|
+-------------------+------+----+-----+
only showing top 5 rows



In [ ]:
entries_gender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: string (nullable = true)
 |-- Total: string (nullable = true)



In [ ]:
entries_gender = entries_gender.withColumn('Female', col('Female').cast(IntegerType()))\
    .withColumn('Male', col('Male').cast(IntegerType()))\
        .withColumn('Total', col('Total').cast(IntegerType()))

In [ ]:
entries_gender.printSchema() 

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
medals.show(5)

+----+--------------------+----+------+------+-----+-------------+
|Rank|            Team/NOC|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
+----+--------------------+----+------+------+-----+-------------+
only showing top 5 rows



In [ ]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team/NOC: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [ ]:
# Top 5 countries with highest no of silver medals
top_silver_countries = medals.orderBy('Silver', ascending = False).select("Team/NOC", 'Silver').show(5)



+--------------------+------+
|            Team/NOC|Silver|
+--------------------+------+
|United States of ...|    41|
|People's Republic...|    32|
|                 ROC|    28|
|       Great Britain|    21|
|               Japan|    14|
+--------------------+------+
only showing top 5 rows



In [ ]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entries_gender.withColumn(
    'Avg_Female', entries_gender['Female'] / entries_gender['Total']
).withColumn(
    'Avg_Male', entries_gender['Male'] / entries_gender['Total']
)
average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [ ]:
athletes.repartition(1).write.mode('overwrite').option('header','true').csv("/mnt/tokyoolymic/transformed-data/athletes")

In [ ]:
medals.repartition(1).write.mode('overwrite').option('header','true').csv("/mnt/tokyoolymic/transformed-data/medals")
coaches.repartition(1).write.mode('overwrite').option('header','true').csv("/mnt/tokyoolymic/transformed-data/coaches")
entries_gender.repartition(1).write.mode('overwrite').option('header','true').csv("/mnt/tokyoolymic/transformed-data/entries_gender")
teams.repartition(1).write.mode('overwrite').option('header','true').csv("/mnt/tokyoolymic/transformed-data/teams")